# Comparative Linguistic Analysis of bioRxiv and PMC

This notebook is a copy of 02_corpora_comparison.ipynb where PLOS Biology reviewers requested that I rerun this analysis without special characters like ('%', '-', '\t', etc.).

In [1]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict, Counter
import csv
import itertools
from pathlib import Path

import numpy as np
import pandas as pd
import pickle
import spacy
from scipy.stats import chi2_contingency
from tqdm import tqdm_notebook

from annorxiver_modules.corpora_comparison_helper import (
    aggregate_word_counts,
    dump_to_dataframe,
    get_term_statistics,
    KL_divergence,
)

# Full Text Comparison (Global)

## Gather Word Frequencies

In [2]:
biorxiv_count_path = Path("output/total_word_counts/biorxiv_total_count.tsv")
pmc_count_path = Path("output/total_word_counts/pmc_total_count.tsv")
nytac_count_path = Path("output/total_word_counts/nytac_total_count.tsv")

In [3]:
if not biorxiv_count_path.exists():
    biorxiv_corpus_count = aggregate_word_counts(
        list(Path("output/biorxiv_word_counts").rglob("*tsv"))
    )
    dump_to_dataframe(biorxiv_corpus_count, "output/biorxiv_total_count.tsv")
    biorxiv_corpus_count.most_common(10)

In [4]:
if not pmc_count_path.exists():
    pmc_corpus_count = aggregate_word_counts(
        list(Path("../../pmc/pmc_corpus/pmc_word_counts").rglob("*tsv"))
    )
    dump_to_dataframe(pmc_corpus_count, "output/pmc_total_count.tsv")
    pmc_corpus_count.most_common(10)

In [5]:
if not nytac_count_path.exists():
    nytac_corpus_count = aggregate_word_counts(
        list(Path("../../nytac/corpora_stats/output").rglob("*tsv"))
    )
    dump_to_dataframe(nytac_corpus_count, "output/nytac_total_count.tsv")
    nytac_corpus_count.most_common(10)

In [6]:
biorxiv_total_count_df = pd.read_csv(biorxiv_count_path.resolve(), sep="\t")

pmc_total_count_df = pd.read_csv(pmc_count_path.resolve(), sep="\t")

nytac_total_count_df = pd.read_csv(nytac_count_path.resolve(), sep="\t")

In [7]:
biorxiv_sentence_length = pickle.load(open("output/biorxiv_sentence_length.pkl", "rb"))
pmc_sentence_length = pickle.load(
    open("../../pmc/pmc_corpus/pmc_sentence_length.pkl", "rb")
)
nytac_sentence_length = pickle.load(
    open("../../nytac/corpora_stats/nytac_sentence_length.pkl", "rb")
)

In [8]:
spacy_nlp = spacy.load("en_core_web_sm")
stop_word_list = list(spacy_nlp.Defaults.stop_words)

## Get Corpora Comparison Stats

In [9]:
biorxiv_sentence_len_list = list(biorxiv_sentence_length.items())
biorxiv_data = {
    "document_count": len(biorxiv_sentence_length),
    "sentence_count": sum(map(lambda x: len(x[1]), biorxiv_sentence_len_list)),
    "token_count": biorxiv_total_count_df["count"].sum(),
    "stop_word_count": (
        biorxiv_total_count_df.query(f"lemma in {stop_word_list}")["count"].sum()
    ),
    "avg_document_length": np.mean(
        list(map(lambda x: len(x[1]), biorxiv_sentence_len_list))
    ),
    "avg_sentence_length": np.mean(
        list(itertools.chain(*list(map(lambda x: x[1], biorxiv_sentence_len_list))))
    ),
    "negatives": (biorxiv_total_count_df.query("dep_tag =='neg'")["count"].sum()),
    "coordinating_conjunctions": (
        biorxiv_total_count_df.query("dep_tag =='cc'")["count"].sum()
    ),
    "coordinating_conjunctions%": (
        biorxiv_total_count_df.query("dep_tag =='cc'")["count"].sum()
    )
    / biorxiv_total_count_df["count"].sum(),
    "pronouns": (biorxiv_total_count_df.query("pos_tag =='PRON'")["count"].sum()),
    "pronouns%": (biorxiv_total_count_df.query("pos_tag =='PRON'")["count"].sum())
    / biorxiv_total_count_df["count"].sum(),
    "passives": (
        biorxiv_total_count_df.query(
            "dep_tag in ['auxpass', 'nsubjpass', 'csubjpass']"
        )["count"].sum()
    ),
    "passive%": (
        biorxiv_total_count_df.query(
            "dep_tag in ['auxpass', 'nsubjpass', 'csubjpass']"
        )["count"].sum()
    )
    / biorxiv_total_count_df["count"].sum(),
}

In [10]:
pmc_sentence_len_list = list(pmc_sentence_length.items())
pmc_data = {
    "document_count": len(pmc_sentence_length),
    "sentence_count": sum(map(lambda x: len(x[1]), pmc_sentence_len_list)),
    "token_count": pmc_total_count_df["count"].sum(),
    "stop_word_count": (
        pmc_total_count_df.query(f"lemma in {stop_word_list}")["count"].sum()
    ),
    "avg_document_length": np.mean(
        list(map(lambda x: len(x[1]), pmc_sentence_len_list))
    ),
    "avg_sentence_length": np.mean(
        list(itertools.chain(*list(map(lambda x: x[1], pmc_sentence_len_list))))
    ),
    "negatives": (pmc_total_count_df.query("dep_tag =='neg'")["count"].sum()),
    "coordinating_conjunctions": (
        pmc_total_count_df.query("dep_tag =='cc'")["count"].sum()
    ),
    "coordinating_conjunctions%": (
        pmc_total_count_df.query("dep_tag =='cc'")["count"].sum()
    )
    / pmc_total_count_df["count"].sum(),
    "pronouns": (pmc_total_count_df.query("pos_tag =='PRON'")["count"].sum()),
    "pronouns%": (pmc_total_count_df.query("pos_tag =='PRON'")["count"].sum())
    / pmc_total_count_df["count"].sum(),
    "passives": (
        pmc_total_count_df.query("dep_tag in ['auxpass', 'nsubjpass', 'csubjpass']")[
            "count"
        ].sum()
    ),
    "passive%": (
        pmc_total_count_df.query("dep_tag in ['auxpass', 'nsubjpass', 'csubjpass']")[
            "count"
        ].sum()
    )
    / pmc_total_count_df["count"].sum(),
}

In [11]:
nytac_sentence_len_list = list(nytac_sentence_length.items())
nytac_data = {
    "document_count": len(nytac_sentence_length),
    "sentence_count": sum(map(lambda x: len(x[1]), nytac_sentence_len_list)),
    "token_count": nytac_total_count_df["count"].sum(),
    "stop_word_count": (
        nytac_total_count_df.query(f"lemma in {stop_word_list}")["count"].sum()
    ),
    "avg_document_length": np.mean(
        list(map(lambda x: len(x[1]), nytac_sentence_len_list))
    ),
    "avg_sentence_length": np.mean(
        list(itertools.chain(*list(map(lambda x: x[1], nytac_sentence_len_list))))
    ),
    "negatives": (nytac_total_count_df.query("dep_tag =='neg'")["count"].sum()),
    "coordinating_conjunctions": (
        nytac_total_count_df.query("dep_tag =='cc'")["count"].sum()
    ),
    "coordinating_conjunctions%": (
        nytac_total_count_df.query("dep_tag =='cc'")["count"].sum()
    )
    / nytac_total_count_df["count"].sum(),
    "pronouns": (nytac_total_count_df.query("pos_tag =='PRON'")["count"].sum()),
    "pronouns%": (nytac_total_count_df.query("pos_tag =='PRON'")["count"].sum())
    / nytac_total_count_df["count"].sum(),
    "passives": (
        nytac_total_count_df.query("dep_tag in ['auxpass', 'nsubjpass', 'csubjpass']")[
            "count"
        ].sum()
    ),
    "passive%": (
        nytac_total_count_df.query("dep_tag in ['auxpass', 'nsubjpass', 'csubjpass']")[
            "count"
        ].sum()
    )
    / nytac_total_count_df["count"].sum(),
}

In [12]:
# This dataframe contains document statistics for each Corpus
# document count - the number of documents within the corpus
# Sentence count - the number of sentences within the corpus
# Token count - the number of tokens within the corpus
# Stop word counts - the number of stop words within the corpus
# Average document length - the average number of sentences within a document for a given corpus
# Average sentence length - the average number of words within a sentence for a given corpus
# Negatives - the number of negations (e.g. placing not in within a sentence) within a given corpus
# Coordinating Conjunctions - the number of coordinating conjunctions (and, but, for etc.) within a given corpus
# Pronouns - the number of pronouns within a given corpus
# Passive - the number of passive words within a given corpus

token_stats_df = pd.DataFrame.from_records(
    [biorxiv_data, pmc_data, nytac_data], index=["bioRxiv", "PMC", "NYTAC"]
).T
token_stats_df.to_csv("output/figures/corpora_token_stats.tsv", sep="\t")
token_stats_df

,bioRxiv,PMC,NYTAC
document_count,7.111800e+04,1.977647e+06,1.855658e+06
sentence_count,2.219574e+07,4.804898e+08,7.217104e+07
token_count,4.209699e+08,8.597101e+09,1.218673e+09
stop_word_count,1.584294e+08,3.153077e+09,5.593911e+08
avg_document_length,3.120973e+02,2.429604e+02,3.889242e+01
avg_sentence_length,2.270775e+01,2.146228e+01,1.989098e+01
negatives,1.148382e+06,2.492880e+07,7.272401e+06
coordinating_conjunctions,1.429574e+07,3.070823e+08,3.873005e+07
coordinating_conjunctions%,3.395904e-02,3.571929e-02,3.178050e-02
pronouns,4.604432e+06,7.499412e+07,4.671255e+07


## LogLikelihood + Odds Ratio + KL Divergence Calculations

The goal here is to compare word frequencies between bioRxiv and pubmed central. The problem when comparing word frequencies is that non-meaningful words (aka stopwords) such as the, of, and, be, etc., appear the most often. To account for this problem the first step here is to remove those words from analyses.

### Remove Stop words

In [13]:
biorxiv_total_count_df = (
    biorxiv_total_count_df.groupby("lemma")
    .agg({"count": "sum"})
    .reset_index()
    .sort_values("count", ascending=False)
)
biorxiv_total_count_df

,lemma,count
2681961,the,22645292
2018157,of,14639457
656569,and,11981201
1572392,in,10132319
2704461,to,8146317
...,...,...
1542255,i.3sl,1
1542254,i.34,1
401503,478052_mir,1
401504,478090_mir,1


In [14]:
pmc_total_count_df = (
    pmc_total_count_df.groupby("lemma")
    .agg({"count": "sum"})
    .reset_index()
    .sort_values("count", ascending=False)
)
pmc_total_count_df

,lemma,count
92606744,the,455469360
76187309,of,305683746
38780140,and,258669384
62245336,in,209051122
93273610,to,154688271
...,...,...
36210554,a2.7106faslfas,1
36210553,a2.710.47appropriateness3.500.710.500.00n,1
36210552,a2.710.014g,1
36210551,"a2.71.62.02.0f(3,139",1


In [15]:
nytac_total_count_df = (
    nytac_total_count_df.groupby("lemma")
    .agg({"count": "sum"})
    .reset_index()
    .sort_values("count", ascending=False)
)
nytac_total_count_df

,lemma,count
1924366,the,72975471
1614479,of,33885977
781698,a,30376684
820301,and,29992626
1937587,to,29018148
...,...,...
1067648,creduto,1
1067650,creeblies,1
1067652,creechan,1
1067660,creedan,1


### Calculate LogLikelihoods and Odds ratios

In [16]:
biorxiv_vs_pmc = get_term_statistics(biorxiv_total_count_df, pmc_total_count_df, 100)

biorxiv_vs_pmc.to_csv(
    "output/comparison_stats/biorxiv_vs_pmc_comparison_special_chars_removed.tsv",
    sep="\t",
    index=False,
)

biorxiv_vs_pmc

,lemma,corpus_one_a,corpus_two_b,corpus_one_c,corpus_two_d,log_likelihood,odds_ratio
0,system,229194,5116317,420924101,8596698154,1782.156189,0.914900
1,sample,253603,4790248,420924101,8596698154,1434.901001,1.081245
2,obtained,187797,4807957,420924101,8596698154,9899.691711,0.797730
3,disease,156577,5089904,420924101,8596698154,38098.929871,0.628270
4,compared,351431,8527264,420924101,8596698154,10563.406982,0.841702
...,...,...,...,...,...,...,...
123,cancer,157764,5656957,420924101,8596698154,58398.580200,0.569578
124,showed,240573,6355817,420924101,8596698154,16640.221924,0.773043
125,described,220661,4419142,420924101,8596698154,80.283447,1.019801
126,respectively,225542,5408486,420924101,8596698154,5860.625793,0.851686


In [17]:
biorxiv_vs_nytac = get_term_statistics(
    biorxiv_total_count_df, nytac_total_count_df, 100
)
biorxiv_vs_nytac.to_csv(
    "output/comparison_stats/biorxiv_nytac_comparison_special_chars_removed.tsv",
    sep="\t",
    index=False,
)
biorxiv_vs_nytac

,lemma,corpus_one_a,corpus_two_b,corpus_one_c,corpus_two_d,log_likelihood,odds_ratio
0,system,229194,411423,420924101,1218668568,31884.849548,1.612859
1,season,14253,544258,420924101,1218668568,228950.505646,0.075820
2,sample,253603,17059,420924101,1218668568,572359.534927,43.041028
3,city,6200,1357957,420924101,1218668568,743010.275284,0.013219
4,compared,351431,98796,420924101,1218668568,540349.911285,10.298703
...,...,...,...,...,...,...,...
187,described,220661,144832,420924101,1218668568,195142.788895,4.411059
188,best,82201,501641,420924101,1218668568,46645.993027,0.474423
189,good,39088,683489,420924101,1218668568,207736.198685,0.165575
190,respectively,225542,25400,420924101,1218668568,463856.934753,25.708428


In [18]:
pmc_vs_nytac = get_term_statistics(pmc_total_count_df, nytac_total_count_df, 100)

pmc_vs_nytac.to_csv(
    "output/comparison_stats/pmc_nytac_comparison_special_chars_removed.tsv",
    sep="\t",
    index=False,
)

pmc_vs_nytac

,lemma,corpus_one_a,corpus_two_b,corpus_one_c,corpus_two_d,log_likelihood,odds_ratio
0,system,5116317,411423,8596698154,1218668568,1.439762e+05,1.762880
1,season,340323,544258,8596698154,1218668568,1.182061e+06,0.088642
2,sample,4790248,17059,8596698154,1218668568,1.114433e+06,39.806900
3,obtained,4807957,29724,8596698154,1218668568,1.036518e+06,22.930169
4,disease,5089904,110870,8596698154,1218668568,7.392109e+05,6.508029
...,...,...,...,...,...,...,...
188,described,4419142,144832,8596698154,1218668568,4.913599e+05,4.325411
189,best,1323985,501641,8596698154,1218668568,2.972458e+05,0.374148
190,good,1437700,683489,8596698154,1218668568,5.663380e+05,0.298188
191,respectively,5408486,25400,8596698154,1218668568,1.216344e+06,30.185330


## Calculate KL Divergence

In [19]:
term_grid = [100, 200, 300, 400, 500, 1000, 1500, 2000, 3000, 5000]
kl_data = []
for num_terms in tqdm_notebook(term_grid):
    kl_data.append(
        {
            "num_terms": num_terms,
            "KL_divergence": KL_divergence(
                biorxiv_total_count_df, pmc_total_count_df, num_terms=num_terms
            ),
            "comparison": "biorxiv_vs_pmc",
        }
    )

    kl_data.append(
        {
            "num_terms": num_terms,
            "KL_divergence": KL_divergence(
                biorxiv_total_count_df, nytac_total_count_df, num_terms=num_terms
            ),
            "comparison": "biorxiv_vs_nytac",
        }
    )

    kl_data.append(
        {
            "num_terms": num_terms,
            "KL_divergence": KL_divergence(
                pmc_total_count_df, nytac_total_count_df, num_terms=num_terms
            ),
            "comparison": "pmc_vs_nytac",
        }
    )

In [20]:
kl_metrics = pd.DataFrame.from_records(kl_data)
kl_metrics.to_csv(
    "output/comparison_stats/corpora_kl_divergence_special_chars_removed.tsv",
    sep="\t",
    index=False,
)
kl_metrics

,num_terms,KL_divergence,comparison
0,100,0.022705,biorxiv_vs_pmc
1,100,0.209552,biorxiv_vs_nytac
2,100,0.819815,pmc_vs_nytac
3,200,0.030102,biorxiv_vs_pmc
4,200,1.156903,biorxiv_vs_nytac
5,200,1.110685,pmc_vs_nytac
6,300,0.036384,biorxiv_vs_pmc
7,300,1.484073,biorxiv_vs_nytac
8,300,1.298867,pmc_vs_nytac
9,400,0.042423,biorxiv_vs_pmc


# Preprint to Published View

In [21]:
mapped_doi_df = (
    pd.read_csv("../journal_tracker/output/mapped_published_doi.tsv", sep="\t")
    .query("published_doi.notnull()")
    .query("pmcid.notnull()")
    .groupby("preprint_doi")
    .agg(
        {
            "author_type": "first",
            "heading": "first",
            "category": "first",
            "document": "first",
            "preprint_doi": "last",
            "published_doi": "last",
            "pmcid": "last",
        }
    )
    .reset_index(drop=True)
)
mapped_doi_df.tail()

,author_type,heading,category,document,preprint_doi,published_doi,pmcid
30922,regular article,new results,microbiology,872325_v1.xml,10.1101/872325,10.1128/mbio.03197-19,PMC7078482
30923,regular article,new results,cell biology,872408_v1.xml,10.1101/872408,10.1186/s13072-020-00335-x,PMC7057672
30924,regular article,new results,ecology,872549_v1.xml,10.1101/872549,10.1093/aob/mcaa101,PMC7539359
30925,regular article,new results,biochemistry,872879_v1.xml,10.1101/872879,10.1038/s41467-020-14898-6,PMC7048817
30926,regular article,new results,developmental biology,873232_v1.xml,10.1101/873232,10.1534/g3.119.400967,PMC7056964


In [22]:
print(f"Total # of Preprints Mapped: {mapped_doi_df.shape[0]}")
print(f"Total % of Mapped: {mapped_doi_df.shape[0]/71118}")

Total # of Preprints Mapped: 30927
Total % of Mapped: 0.43486880958407154


In [34]:
preprint_count = aggregate_word_counts(
    [
        Path(f"output/biorxiv_word_counts/{Path(file)}.tsv")
        for file in mapped_doi_df.document.values.tolist()
        if Path(f"output/biorxiv_word_counts/{Path(file)}.tsv").exists()
    ]
)

preprint_count_df = (
    pd.DataFrame.from_records(
        [
            {
                "lemma": token[0],
                "pos_tag": token[1],
                "dep_tag": token[2],
                "count": preprint_count[token],
            }
            for token in preprint_count
        ]
    )
    .groupby("lemma")
    .agg({"count": "sum"})
    .reset_index()
    .sort_values("count", ascending=False)
)

preprint_count_df.head()

,lemma,count
1475592,the,10336972
1109828,of,6697661
362762,and,5443114
863927,in,4638238
1487920,to,3757477


In [35]:
published_count = aggregate_word_counts(
    [
        Path(f"../../pmc/pmc_corpus/pmc_word_counts/{file}.tsv")
        for file in mapped_doi_df.pmcid.values.tolist()
        if Path(f"../../pmc/pmc_corpus/pmc_word_counts/{file}.tsv").exists()
    ]
)

published_count_df = (
    pd.DataFrame.from_records(
        [
            {
                "lemma": token[0],
                "pos_tag": token[1],
                "dep_tag": token[2],
                "count": published_count[token],
            }
            for token in published_count
        ]
    )
    .groupby("lemma")
    .agg({"count": "sum"})
    .reset_index()
    .sort_values("count", ascending=False)
)

published_count_df.head()

,lemma,count
2113302,the,6818238
1654790,of,4378340
659905,and,3602889
1317069,in,2996432
2130676,to,2382655


In [36]:
preprint_vs_published = get_term_statistics(preprint_count_df, published_count_df, 100)

preprint_vs_published.to_csv(
    "output/comparison_stats/preprint_to_published_comparison_special_chars_removed.tsv",
    sep="\t",
    index=False,
)

preprint_vs_published

,lemma,corpus_one_a,corpus_two_b,corpus_one_c,corpus_two_d,log_likelihood,odds_ratio
0,reads,88790,68060,193427969,130362488,633.838278,0.879236
1,system,99703,61713,193427969,130362488,277.788486,1.088842
2,sample,118574,82765,193427969,130362488,59.730513,0.965553
3,compared,158707,101852,193427969,130362488,149.063377,1.050171
4,mean,133827,95028,193427969,130362488,150.905439,0.949130
...,...,...,...,...,...,...,...
101,mice,129553,69514,193427969,130362488,2399.743786,1.256055
102,showed,102901,63549,193427969,130362488,301.723806,1.091300
103,described,107724,75203,193427969,130362488,54.740002,0.965408
104,respectively,101772,67963,193427969,130362488,3.438385,1.009228


Main takeaways from this analysis:
1. On a global scale bioRxiv contains more field specific articles as top words consist of: neuron, gene, genome, network
2. "Patients" appear more correlated with PMC as most preprints involving patients are shipped over to medRxiv.
3. Many words associated with PMC are health related which ties back to the medRxiv note.
4. Citation styles change as preprints transition to published versions. Et Al. has a greater association within bioRxiv compared to PMC.
5. On a local scale published articles contain more statistical concepts (e.g., t-test) as well as quantitative measures (e.g. degree signs). (High associated lemmas are t, -, degree sign etc.)
6. Publish articles have a focus shift on mentioning figures, adding supplementary data etc compared to preprints.
7. Preprints have a universal way of citing published works by using the et al. citation. Hard to pinpoint if leading factor is because of peer review or journal style, but it will be an interesting point to discuss in the paper.